In [2]:
import pandas as pd
import os
import gzip
import spacy as sp
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from math import log
from collections import defaultdict
from sklearn.cluster import DBSCAN
from math import log
from nltk.corpus import wordnet as wn
import nltk
from itertools import product
import pickle
import warnings
warnings.filterwarnings('ignore')

In [44]:
nlp = sp.load('en')

In [1]:
eval_data = pd.read_csv("eval-mod.csv")

metaphor_words = []
metaphors = {}

for r in eval_data.iterrows():
    tokens = nlp(r[1]["Metaphor"].decode('utf-8', errors='ignore').strip().lower())
    topic = tokens[0]
    vehicle = tokens[-1]
    if topic.lemma_ not in metaphor_words:
        metaphor_words.append(topic.lemma_)
    if vehicle.lemma_ not in metaphor_words:
        metaphor_words.append(vehicle.lemma_)
    if r[1]["Metaphor"] not in metaphors:
        metaphors[r[1]["Metaphor"]] = [topic.lemma_, vehicle.lemma_, 
                                       [(str(r[1]["Interpretation"]).lower(), r[1]["Freq"])],
                                       [str(r[1]["Interpretation"]).lower()]
                                      ]
    else:
        metaphors[r[1]["Metaphor"]][2].append((str(r[1]["Interpretation"]).lower(), r[1]["Freq"]))
        metaphors[r[1]["Metaphor"]][3].append(str(r[1]["Interpretation"]).lower())


NameError: name 'pd' is not defined

In [162]:
class Lang:
    def __init__(self):
        self.vec = []
        self.word_count = 0
        self.ind2word = {}
        self.word2ind = {}
        for line in open("../glove.6B/glove.6B.300d.txt"):
            values = line.split(" ")
            v = []
            for i in range (1, len(values)):
                v.append(float(values[i]))
            self.vec.append(v)
            self.ind2word[self.word_count] = values[0]
            self.word2ind[values[0]] = self.word_count
            self.word_count += 1
    
    def get_vec(self, word):
        word = word.strip().lower()
        if word in self.word2ind:
            return self.vec[self.word2ind[word]]
        return None
            
lang = Lang()

In [371]:
class DepNode:
    def __init__(self, line):
        tokens = line.split('\t')
        self.id = int(tokens[0])
        self.text = tokens[1].lower()
        self.lemma = tokens[2].lower()
        self.pos = tokens[3]
        self.head = int(tokens[6])
        self.dep_type = tokens[7]
        self.children = []

class DepSentence:
    def __init__(self):
        self.nodes = []
        
    def add_word(self, line):
        node = DepNode(line)
        self.nodes.append(node)
        return node
        
    def rewire(self):
        for n in self.nodes:
            if n.dep_type != 'ROOT':
                self.nodes[n.head].children.append(n)
    
    def is_valid_meaning(self):
        #if self.pos in ['JJ', 'JJS', 'RB', 'VBG']:
        if self.pos in ['JJ', 'JJS', 'VBG']:
            return True
        return False
    
    
def find_candidates_for(words, dep_corpus_folder):
    candidates = {}
    files = [f for f in os.listdir(dep_corpus_folder) if os.path.isfile(os.path.join(dep_corpus_folder, f))]
    for f in files:
        full_path = os.path.join(dep_corpus_folder, f)
        print "processing", full_path
        sentence = None
        valid = False
        for line in gzip.open(full_path):
            line = line.strip()
            if len(line) > 0 and line[0] != '#':
                if sentence is None:
                    sentence = DepSentence()
                node = sentence.add_word(line)
                if node.text in words:
                    valid = True
            elif sentence is not None:
                if valid:
                    sentence.rewire()
                    for n in sentence.nodes:
                        if n.text in words:
                            if n.text not in candidates:
                                candidates[n.text] = Counter()
                            if n.dep_type != 'ROOT' and is_valid_meaning(sentence.nodes[n.head].pos):
                                candidates[n.text][sentence.nodes[n.head].text] += 1
                            for c in n.children:
                                if is_valid_meaning(c.pos):
                                    candidates[n.text][c.text] += 1
                sentence = None
    return candidates

In [372]:
candidates = find_candidates_for(metaphor_words, 'dep_corpus')

processing dep_corpus/part-m-00000.gz
processing dep_corpus/part-m-00001.gz
processing dep_corpus/part-m-00002.gz
processing dep_corpus/part-m-00003.gz
processing dep_corpus/part-m-00004.gz
processing dep_corpus/part-m-00005.gz
processing dep_corpus/part-m-00006.gz
processing dep_corpus/part-m-00007.gz
processing dep_corpus/part-m-00008.gz
processing dep_corpus/part-m-00009.gz
processing dep_corpus/part-m-00010.gz
processing dep_corpus/part-m-00011.gz
processing dep_corpus/part-m-00012.gz
processing dep_corpus/part-m-00013.gz
processing dep_corpus/part-m-00014.gz
processing dep_corpus/part-m-00015.gz
processing dep_corpus/part-m-00016.gz
processing dep_corpus/part-m-00017.gz
processing dep_corpus/part-m-00018.gz


In [478]:
pickle.dump(candidates, open( "candidates.p", "wb" ) )

In [373]:
print candidates['god'].most_common(200)

[('good', 333), ('not', 297), ('only', 228), ('true', 116), ('loving', 102), ('alone', 98), ('same', 89), ('angry', 75), ('great', 74), ('willing', 72), ('personal', 71), ("n't", 70), ('dear', 70), ('alive', 68), ('almighty', 67), ('all-powerful', 65), ('dead', 61), ('real', 54), ('just', 51), ('other', 50), ('faithful', 49), ('even', 48), ('then', 47), ('no', 44), ('merciful', 43), ('powerful', 41), ('so', 39), ('ever-living', 39), ('much', 38), ('such', 37), ('perfect', 37), ('omniscient', 37), ('present', 37), ('able', 35), ('omnipotent', 35), ('new', 34), ('false', 34), ('own', 34), ('holy', 34), ('infinite', 31), ('eternal', 28), ('also', 28), ('there', 28), ('responsible', 26), ('first', 24), ('perhaps', 24), ('amazing', 23), ('jealous', 23), ('human', 23), ('particular', 21), ('now', 21), ('best', 21), ('supernatural', 21), ('different', 21), ('happy', 20), ('awful', 19), ('christian', 19), ('therefore', 19), ('awesome', 19), ('wrong', 19), ('fully', 19), ('evil', 19), ('big', 1